## Install folium to create maps

In [1]:
## Install the tools necessary
!conda install -c conda-forge folium=0.5.0 --yes 

print ("Folium and geopy installed")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:          4.1.0-py_1        conda-forge
    branca:          0.4.0-py_0        conda-forge
    folium:          0.5.0-py_0        conda-forge
    python_abi:      3.6-1_cp36m       conda-forge
    vincent:      

## Import libraries

In [68]:
## BeautifulSoup to scrape data 
from bs4 import BeautifulSoup

##urllib.requestto open URLs
from urllib.request import urlopen

# requests to handle requests
import requests

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# numpy to handle data in a vectorized manner
import numpy as np

# pandas for data analsysis and allow the frames to have as many lines as needed
import pandas as pd
pd.set_option('display.max_rows', None)

# json to handle JSON files
import json 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means for clustering stage
from sklearn.cluster import KMeans

# folium for rendering library
import folium

print('Libraries imported.')

Libraries imported.


## Get the data

Scrape the data for Toronto neighbourhoods from wikipedia
Remove the lines where no Borough was assigned
Get the data from Geospatial from a csv to collect the latitude and longitude for each postcode
Merge the toronto neighbourhood dataframe with the latitude and longitude dataframe

In [69]:
## Get the data

with urlopen('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050') as response:
    soup = BeautifulSoup(response, 'html.parser')

table = soup.find_all('table')
df = pd.read_html(str(table))[0]

## Remove lines where the Borough is not assigned
df = df[df.Borough != 'Not assigned']

## Load the Geospatial_data into a dataframe to extract the latitude and longitude

postcode_data = pd.read_csv('http://cocl.us/Geospatial_data')

## Rename the column
postcode_data.rename(columns={'Postal Code': 'Postcode'}, inplace=True)

## Merge the information from the csv file into the main dataframe
df = pd.merge(df,
                 postcode_data[['Postcode', 'Latitude', 'Longitude']],
                 on='Postcode')
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


## Explore the neighbourhoods in North York, Toronto

Create a new dataframe with the neighbourhoods from North York only

In [104]:
df_toronto = df[df["Borough"].str.contains("North York")]

### Create a map with the neighborhoods in North York

In [106]:
address = 'North York, Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.7543263, -79.44911696639593.


### Explore venues per Neighbourhood in North York

Create a function that send requests to the Foursquare API to get the nearby venues for each neighbourhood in a radius of 500 miles

In [72]:
##Function to get the venues by neighbourhood

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [73]:
##Foursquare credentials

CLIENT_ID = 'YUOJ52TP30ZA1BIAOW1VRNNDURAUIMUIT4K2HG3SLMHK4S5M'
CLIENT_SECRET = 'KQQHKVWPY0ETBI3RQKCRGDWTBBBLJAII3XBDV4HX4W0HHR4W'
VERSION = '20180605' # Foursquare API version

### Creates a new data frame with the neighbourhoods in North York with its laitutde and longitude

In [74]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Heights
Lawrence Manor
Don Mills North
Glencairn
Flemingdon Park
Don Mills South
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Fairview
Henry Farm
Oriole
Northwood Park
York University
Bayview Village
CFB Toronto
Downsview East
Silver Hills
York Mills
Downsview West
Downsview
North Park
Upwood Park
Humber Summit
Newtonbrook
Willowdale
Downsview Central
Bedford Park
Lawrence Manor East
Emery
Humberlea
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


In [107]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [76]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Bathurst Manor,20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
Bedford Park,25,25,25,25,25,25
CFB Toronto,3,3,3,3,3,3
Don Mills North,5,5,5,5,5,5
Don Mills South,21,21,21,21,21,21
Downsview,4,4,4,4,4,4
Downsview Central,2,2,2,2,2,2
Downsview East,3,3,3,3,3,3


## Create categories for the venues

Create a new dataframe with the neighbourhoods and categories
Group the new dataframe to count the amount of venus per venue type

In [110]:

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

toronto_grouped.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,Bathurst Manor,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.10,0.0,...,0.0,0.05,0.0,0.00,0.0,0.0,0.0,0.05,0.0,0.0
1,Bayview Village,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.25,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0
2,Bedford Park,0.0,0.000000,0.04,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.04,0.0,0.08,0.0,0.0,0.0,0.00,0.0,0.0
3,CFB Toronto,0.0,0.333333,0.00,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0
4,Don Mills North,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0


### Calculates the five most popular venues per neighbourhood

In [112]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor----
           venue  freq
0    Coffee Shop  0.10
1           Bank  0.10
2  Shopping Mall  0.05
3    Supermarket  0.05
4  Deli / Bodega  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.08
2          Restaurant  0.08
3      Sandwich Place  0.08
4     Thai Restaurant  0.08


----CFB Toronto----
                           venue  freq
0                        Airport  0.33
1                           Park  0.33
2                       Bus Stop  0.33
3                   Liquor Store  0.00
4  Paper / Office Supplies Store  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center   0.2
1  Caribbean Restaurant   0.2
2                  Café   0.2
3        Baseball Field   0.2
4   Japanese Res

In [80]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [114]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhood_venues_sorted = pd.DataFrame(columns=columns)
neighbourhood_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhood_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bathurst Manor,Coffee Shop,Bank,Diner,Shopping Mall,Middle Eastern Restaurant,Pharmacy,Pizza Place,Deli / Bodega,Ice Cream Shop,Bridal Shop
1,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank,Women's Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
2,Bedford Park,Sandwich Place,Thai Restaurant,Italian Restaurant,Restaurant,Coffee Shop,Café,Liquor Store,Juice Bar,Fast Food Restaurant,Butcher
3,CFB Toronto,Airport,Bus Stop,Park,Women's Store,Discount Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
4,Don Mills North,Japanese Restaurant,Caribbean Restaurant,Gym / Fitness Center,Café,Baseball Field,Women's Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega


## Cluster the neighbourhoods

Cluster the neighbourhoods in 5 clusters

In [116]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 4, 1, 1, 0, 2, 4, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 2, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1], dtype=int32)

### Create a new dataframe with the cluster labels

In [117]:
## create a copy of the original dataframe

toronto_merged = df_toronto

##Merge the original dataframe with the dataframa with the most common vanues data

toronto_merged = toronto_merged.join(neighbourhood_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

##Drop the lines with null values as no venues were returned for some neighborhoods

toronto_merged = toronto_merged.dropna()

## Add the cluster lables to the dataframe

toronto_merged['Cluster Labels'] = kmeans.labels_

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M3A,North York,Parkwoods,43.753259,-79.329656,Park,Food & Drink Shop,Discount Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,1
1,M4A,North York,Victoria Village,43.725882,-79.315572,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Intersection,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,1
3,M6A,North York,Lawrence Heights,43.718518,-79.464763,Clothing Store,Furniture / Home Store,Boutique,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Vietnamese Restaurant,Dog Run,Convenience Store,1
4,M6A,North York,Lawrence Manor,43.718518,-79.464763,Clothing Store,Furniture / Home Store,Boutique,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Vietnamese Restaurant,Dog Run,Convenience Store,4
9,M3B,North York,Don Mills North,43.745906,-79.352188,Japanese Restaurant,Caribbean Restaurant,Gym / Fitness Center,Café,Baseball Field,Women's Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,1


### Create a map to visualize the clusters

In [103]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters